In [1]:
import torch
from torch.utils.data import DataLoader, random_split, TensorDataset
import pickle
import torchvision
import torch.nn as nn
import torch.optim as optim
from torchvision.models import resnet34
from torchvision import transforms
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, AdaBoostClassifier
import xgboost as xgb
from sklearn.metrics import accuracy_score
import random

# set random seed
seed = 42
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)
torch.cuda.manual_seed_all(seed)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
DATA_PATH = '/content/drive/My Drive/Colab Notebooks/attack/pickle/tinyimagenet/resnet34/shadow.p'

device=torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

with open(DATA_PATH, "rb") as f:
    dataset = pickle.load(f)
print(type(dataset), len(dataset))


# split the dataset
train_size = int(0.8 * len(dataset))
test_size = len(dataset) - train_size
train_dataset, test_dataset = random_split(dataset, [train_size, test_size])

# create dataloaders
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True, num_workers=2)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False, num_workers=2)

cuda:0
<class 'list'> 50000


# shadow model

In [ ]:
# model parameters
n_classes = 200
n_epochs = 40
batch_size = 64
learning_rate = 0.001
weight_decay = 1e-4  # parameter for L2 regularization

# load target model for shadow model
shadow_model = resnet34(num_classes=200).to(device)
state_dict = torch.load('/content/drive/My Drive/Colab Notebooks/attack/models/resnet34_tinyimagenet.pth', map_location=device)
shadow_model.load_state_dict(state_dict['net'])
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(shadow_model.parameters(), lr=learning_rate, weight_decay=weight_decay)
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=10, gamma=0.1)

# model training
shadow_model.train()
for epoch in range(n_epochs):
    running_loss = 0.0
    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = shadow_model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
    scheduler.step()

    print(f"Epoch {epoch+1}/{n_epochs}, Loss: {running_loss/len(train_loader)}")
    # save the model after each 5 epochs
    if (epoch+1) % 5 == 0:
        torch.save(shadow_model.state_dict(), f"/content/drive/My Drive/Colab Notebooks/attack/my_models/tinyimagenet_resnet/shadow_model_epoch_{epoch+1}.pt")

Epoch 1/40, Loss: 3.8695884323120118
Epoch 2/40, Loss: 3.2373982093811033
Epoch 3/40, Loss: 2.7816393627166747
Epoch 4/40, Loss: 2.293316515159607
Epoch 5/40, Loss: 1.761449604511261
Epoch 6/40, Loss: 1.2288123917579652
Epoch 7/40, Loss: 0.8173454327106476
Epoch 8/40, Loss: 0.5652892118692399
Epoch 9/40, Loss: 0.43818574702739715
Epoch 10/40, Loss: 0.38657752920389177
Epoch 11/40, Loss: 0.15166444813013077
Epoch 12/40, Loss: 0.04426120265126229
Epoch 13/40, Loss: 0.02596762011349201
Epoch 14/40, Loss: 0.018214115408807994
Epoch 15/40, Loss: 0.012986440163105727
Epoch 16/40, Loss: 0.010029142214730382
Epoch 17/40, Loss: 0.00824177646934986
Epoch 18/40, Loss: 0.006954689632356167
Epoch 19/40, Loss: 0.006238756485842168
Epoch 20/40, Loss: 0.006406092872843146
Epoch 21/40, Loss: 0.004803804163075983
Epoch 22/40, Loss: 0.00405293404571712
Epoch 23/40, Loss: 0.0037582494378089904
Epoch 24/40, Loss: 0.003455909496359527
Epoch 25/40, Loss: 0.0033639930360019206
Epoch 26/40, Loss: 0.00313429859

In [6]:
# load trained shadow model
n_classes = 200
criterion = nn.CrossEntropyLoss()

shadow_model = resnet34(num_classes=n_classes).to(device)
shadow_model.load_state_dict(torch.load("/content/drive/My Drive/Colab Notebooks/attack/my_models/tinyimagenet_resnet/shadow_model_epoch_40.pt"))

<All keys matched successfully>

In [7]:
# model evaluation
test_loss = 0.0
correct = 0
total = 0
shadow_model.eval()
with torch.no_grad():
    for inputs, labels in test_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        outputs = shadow_model(inputs)
        loss = criterion(outputs, labels)
        test_loss += loss.item()
        _, predicted = torch.max(outputs.data, 1) # get the class index
        total += labels.size(0) # total samples
        correct += (predicted == labels).sum().item() # correctly predicted samples

print(f'Test Loss: {test_loss/len(test_loader)}, Accuracy: {100 * correct / total}%')

Test Loss: 4.796599676654597, Accuracy: 25.11%


# Attack Model Training

In [8]:
# method for generating attack dataset for given model and dataset
def generate_attack_data(model, data_loader, label):
    attack_data = []
    model.eval()
    with torch.no_grad():
        for inputs, labels in data_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            probabilities = nn.functional.softmax(outputs, dim=1)
            for prob in probabilities:
                attack_data.append((prob.cpu().numpy(), label))
    return attack_data

# generate attack data
member_data = generate_attack_data(shadow_model, train_loader, 1) # member label as 1
non_member_data = generate_attack_data(shadow_model, test_loader, 0) # non-member label as 0
attack_data = member_data + non_member_data

# prepare the data for training attack model
np.random.shuffle(attack_data)
X = np.array([x[0] for x in attack_data])
y = np.array([x[1] for x in attack_data])
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=seed)

In [ ]:
# train attack model

# attack model class
class AttackModel(nn.Module):
    def __init__(self):
        super(AttackModel, self).__init__()
        self.fc1 = nn.Linear(200, 512)
        self.fc2 = nn.Linear(512, 256)
        self.fc3 = nn.Linear(256, 64)
        self.fc4 = nn.Linear(64, 1)
        self.dropout = nn.Dropout(0.2)

    def forward(self, x):
        x = self.dropout(nn.functional.relu(self.fc1(x)))
        x = self.dropout(nn.functional.relu(self.fc2(x)))
        x = self.dropout(nn.functional.relu(self.fc3(x)))
        x = nn.functional.sigmoid(self.fc4(x))
        return x

# attack model instantiation
attack_model_nn = AttackModel().to(device)
criterion = nn.BCELoss()
optimizer = optim.Adam(attack_model_nn.parameters(), lr=0.001)

# prepare the data for the attack model
X_train_tensor = torch.tensor(X_train, dtype=torch.float32).to(device)
y_train_tensor = torch.tensor(y_train, dtype=torch.float32).to(device)
X_test_tensor = torch.tensor(X_test, dtype=torch.float32).to(device)
y_test_tensor = torch.tensor(y_test, dtype=torch.float32).to(device)
train_dataset_attack = TensorDataset(X_train_tensor, y_train_tensor)
train_loader_attack = DataLoader(train_dataset_attack, batch_size=64, shuffle=True)

# training
for epoch in range(10):
    attack_model_nn.train()
    running_loss = 0.0
    for inputs, labels in train_loader_attack:
        optimizer.zero_grad()
        outputs = attack_model_nn(inputs)
        loss = criterion(outputs, labels.view(-1, 1))
        loss.backward()
        optimizer.step()
        running_loss += loss.item()

    running_loss /= len(train_loader_attack)

    print(f"Epoch {epoch+1}/5000, Loss: {running_loss}")

# evaluate the attack model on the test data
attack_model_nn.eval()
with torch.no_grad():
    outputs = attack_model_nn(X_test_tensor)
    y_pred = outputs.round().view(-1).cpu().numpy()
    accuracy = accuracy_score(y_test, y_pred)
    print(f"Attack Model Accuracy: {accuracy}")


Epoch 1/5000, Loss: 0.26058964723348615
Epoch 2/5000, Loss: 0.16954686414003373
Epoch 3/5000, Loss: 0.15395156400501728
Epoch 4/5000, Loss: 0.14564996304810046
Epoch 5/5000, Loss: 0.1391756833344698
Epoch 6/5000, Loss: 0.13553759976625443
Epoch 7/5000, Loss: 0.13248749970793725
Epoch 8/5000, Loss: 0.12837615408599376
Epoch 9/5000, Loss: 0.12509877087771892
Epoch 10/5000, Loss: 0.12337676727473736
Attack Model Accuracy: 0.9672


# Target Model

In [10]:
# load the target model
MODEL_PATH = '/content/drive/My Drive/Colab Notebooks/attack/models/resnet34_tinyimagenet.pth'

device=torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

target_model = resnet34(num_classes=200).to(device)

state_dict = torch.load(MODEL_PATH, map_location=device)
target_model.load_state_dict(state_dict['net'])

print(target_model)

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

In [11]:
# loading evaluation data
with open('/content/drive/My Drive/Colab Notebooks/attack/pickle/tinyimagenet/resnet34/eval.p', 'rb') as f:
    eval_dataset = pickle.load(f)

# prepare the data for the evaluation
eval_images = [item[0] for item in eval_dataset]
eval_labels = [item[1] for item in eval_dataset]
eval_membership_status = [item[2] for item in eval_dataset]

eval_images_tensor = torch.stack(eval_images)
eval_labels_tensor = torch.tensor(eval_labels)
eval_dataset_tensor = TensorDataset(eval_images_tensor, eval_labels_tensor)
eval_loader = DataLoader(eval_dataset_tensor, batch_size=64, shuffle=False, num_workers=2)


In [12]:
#  Generate evaluation data for attack model using target model
target_model.eval()
probabilities = []
with torch.no_grad():
    for inputs, _ in eval_loader:
        inputs = inputs.to(device)
        outputs = target_model(inputs)
        probs = torch.softmax(outputs, dim=1)
        probabilities.extend(probs.cpu().numpy())

X_eval = np.array(probabilities)
y_eval_true = np.array(eval_membership_status)

In [ ]:
# evaluate the attack model
attack_model_nn.eval()
with torch.no_grad():
    outputs = attack_model_nn(torch.tensor(X_eval, dtype=torch.float32).to(device))
    y_pred = outputs.round().view(-1).cpu().numpy()
    accuracy = accuracy_score(y_eval_true, y_pred)
    print(f"Membership Inference Accuracy: {accuracy}")

Membership Inference Accuracy: 0.995


In [14]:
# save attack model
torch.save(attack_model_nn.state_dict(), f"/content/drive/My Drive/Colab Notebooks/attack/attack_models/tinyimagenet_resnet/attack_model_99.pt")